<a href="https://colab.research.google.com/github/samanthajmichael/ml_project/blob/main/notebooks/Base_Model_Training_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install tensorflow ffmpeg-python opencv-python matplotlib

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [4]:
# Check if GPU is available
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

# Get GPU device information
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    for device in gpu_devices:
        # Get device details
        device_details = tf.config.experimental.get_device_details(device)
        print("GPU Details:", device_details)

GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU Details: {'compute_capability': (7, 5), 'device_name': 'Tesla T4'}


In [5]:
def load_and_preprocess_frame(frame_data, target_size=(224, 224)):
    # Your existing preprocessing code remains the same
    nparr = np.frombuffer(frame_data, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img.astype(np.float32) / 255.0
    return tf.convert_to_tensor(img)

@tf.function
def augment_frame(frame):
    # Enhanced data augmentation
    frame = tf.image.random_flip_left_right(frame)
    frame = tf.image.random_brightness(frame, max_delta=0.2)
    frame = tf.image.random_contrast(frame, lower=0.8, upper=1.2)
    frame = tf.image.random_saturation(frame, lower=0.8, upper=1.2)
    # Add random rotation
    frame = tf.image.rot90(frame, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    frame = tf.clip_by_value(frame, 0.0, 1.0)
    return frame

class ContrastiveLearningModel(tf.keras.Model):
    def __init__(self, temperature=0.5):
        super(ContrastiveLearningModel, self).__init__()
        self.temperature = temperature
        self.base_model = self._create_base_model()
        self.loss_tracker = tf.keras.metrics.Mean(name='loss')

    def _create_base_model(self, input_shape=(224, 224, 3)):
        base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
        x = GlobalAveragePooling2D()(base_model.output)
        x = tf.keras.layers.BatchNormalization()(x)
        x = Dense(256, activation='relu')(x)  # Increased projection head size
        x = tf.keras.layers.Dropout(0.3)(x)   # Added dropout
        projection_head = Dense(128, activation=None)(x)
        projection_head = tf.keras.layers.Lambda(
            lambda x: tf.math.l2_normalize(x, axis=1)
        )(projection_head)
        return Model(inputs=base_model.input, outputs=projection_head)

    def call(self, inputs, training=False):
        return self.base_model(inputs, training=training)

    def train_step(self, data):
        x = data
        batch_size = tf.shape(x)[0]

        with tf.GradientTape() as tape:
            # Generate two augmented views
            x_i = augment_frame(x)
            x_j = augment_frame(x)

            # Get embeddings
            z_i = self.base_model(x_i, training=True)
            z_j = self.base_model(x_j, training=True)

            # Compute NT-Xent loss
            loss = self._nt_xent_loss(z_i, z_j)

        # Compute and apply gradients
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # Update metrics
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _nt_xent_loss(self, z_i, z_j):
        # Your existing NT-Xent loss implementation
        z_i = tf.math.l2_normalize(z_i, axis=1)
        z_j = tf.math.l2_normalize(z_j, axis=1)
        similarity_matrix = tf.matmul(z_i, z_j, transpose_b=True) / self.temperature

        batch_size = tf.shape(z_i)[0]
        contrastive_labels = tf.range(batch_size)

        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=contrastive_labels,
            logits=similarity_matrix
        )
        return tf.reduce_mean(loss)

def train_contrastive_model(df, epochs=10, batch_size=32, steps_per_epoch=50):
    # Create dataset
    generator = tf.data.Dataset.from_generator(
        lambda: data_generator(df, batch_size),
        output_signature=(
            tf.TensorSpec(shape=(batch_size, 224, 224, 3), dtype=tf.float32)
        )
    ).prefetch(tf.data.AUTOTUNE)

    # Create model and callbacks
    model = ContrastiveLearningModel()

    # Learning rate scheduler
    lr_scheduler = ReduceLROnPlateau(
        monitor='loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    # Early stopping
    early_stopping = EarlyStopping(
        monitor='loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )

    # TensorBoard callback for monitoring
    tensorboard = TensorBoard(
        log_dir='./logs',
        histogram_freq=1,
        write_graph=True,
        update_freq='epoch'
    )

    # Compile model with cosine decay learning rate
    initial_learning_rate = 0.001
    decay_steps = epochs * steps_per_epoch
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate, decay_steps
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    )
        # Train model
    history = model.fit(
        generator,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        callbacks=[lr_scheduler, early_stopping, tensorboard],
        verbose=1
    )

    return model, history

In [6]:
# Usage
df = pd.read_pickle("/content/drive/MyDrive/ML Project/middle_15min_frames.pkl")

In [7]:
print(df.info())
print(f"DataFrame shape: {df.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20700 entries, 0 to 20699
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   frame_number  20700 non-null  int64 
 1   frame_data    20700 non-null  object
dtypes: int64(1), object(1)
memory usage: 323.6+ KB
None
DataFrame shape: (20700, 2)


In [8]:
# Investigate frame numbers
print("\nFrame number statistics:")
print(df['frame_number'].describe())

# Check for duplicate frame numbers
duplicates = df['frame_number'].duplicated().sum()
print(f"\nNumber of duplicate frame numbers: {duplicates}")

# Display a few rows of the DataFrame
print("\nFirst few rows of the DataFrame:")
print(df.head())


Frame number statistics:
count     20700.000000
mean      92814.500000
std        5975.719622
min       82465.000000
25%       87639.750000
50%       92814.500000
75%       97989.250000
max      103164.000000
Name: frame_number, dtype: float64

Number of duplicate frame numbers: 0

First few rows of the DataFrame:
   frame_number                                         frame_data
0         82465  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1         82466  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2         82467  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3         82468  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4         82469  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


In [9]:
trained_model = train_contrastive_model(df, epochs=4, batch_size=30, steps_per_epoch=50)
trained_model.save('/content/drive/MyDrive/ML Project/trained_base_model_4_epochs.keras')
print("Training completed and model saved.")

Epoch 1/4, Step 0, Loss: 2.9618
Epoch 1/4, Step 10, Loss: 2.2845
Epoch 1/4, Step 20, Loss: 2.1266
Epoch 1/4, Step 30, Loss: 2.2418
Epoch 1/4, Step 40, Loss: 2.0911
Epoch 1/4, Avg Loss: 2.2394
Epoch 2/4, Step 0, Loss: 2.1812
Epoch 2/4, Step 10, Loss: 2.1067
Epoch 2/4, Step 20, Loss: 2.1725
Epoch 2/4, Step 30, Loss: 2.0990
Epoch 2/4, Step 40, Loss: 2.1842
Epoch 2/4, Avg Loss: 2.1657
Epoch 3/4, Step 0, Loss: 2.0956
Epoch 3/4, Step 10, Loss: 2.6612
Epoch 3/4, Step 20, Loss: 2.3105
Epoch 3/4, Step 30, Loss: 2.2161
Epoch 3/4, Step 40, Loss: 2.0678
Epoch 3/4, Avg Loss: 2.1911
Epoch 4/4, Step 0, Loss: 2.1215
Epoch 4/4, Step 10, Loss: 2.0872
Epoch 4/4, Step 20, Loss: 2.0267
Epoch 4/4, Step 30, Loss: 2.1604
Epoch 4/4, Step 40, Loss: 2.0768
Epoch 4/4, Avg Loss: 2.1182
Training completed and model saved.


In [10]:
trained_model

<Functional name=functional, built=True>